In [1]:
from datetime import datetime
from multiprocessing import Pool
import numpy as np
import random
from libGenerator import setProperties
from vSystem import F
from analyseGrammar import branching_turtle_to_coords
from computeVoxel import process_network
from utils import bezier_interpolation
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.io as scio
%matplotlib widget

In [3]:
n = 20
d0mean = 20.0
d0std = 5.0
tissueVolume = (512, 512, 512)

# Default Parameters
vessels = []

pbar = tqdm(total=n)

while len(vessels)<n:
    # print("Creating image ... %i with %i iterations" % (file, niter))
    # Lindenmayer System Parameters
    now = datetime.now()
    timestamp = now.strftime("%Y%m%dT%H%M%S")
    
    properties = {
        "k": random.uniform(1,10),
        "epsilon": random.uniform(4,20), # Proportion between length & diameter
        "randmarg": random.uniform(1,10),  # Randomness margin between length & diameter
        "sigma": random.uniform(1,10),  # Determines type deviation for Gaussian distributions
        "stochparams": True,
        "timestamp": timestamp,
        
    }  # Whether the generated parameters will also be stochastic
    
    d0 = np.random.normal(d0mean, d0std)  # Ramdonly assign base diameter (no dimension)
    niter = random.randint(6, 14)  # Randomly assign number of L-System recursive loops
    setProperties(properties)  # Setting L-Sytem properties
    
    properties.update({
            "d0": d0, 
            "niter": niter,
            "d0mean": d0mean,
            "d0std": d0std,
            "tissueVolume": tissueVolume,
        })

    try:
        
        """ Run L-System grammar for n iterations """
        turtle_program = F(niter, d0)

        """ Convert grammar into coordinates """
        coords = branching_turtle_to_coords(turtle_program, d0)

        """ Analyse / sort coordinate data """
        update = bezier_interpolation(coords)

        """ If you fancy, plot a 2D image of the network! """
        # plot_coords(newdata, array=True, bare_plot=False) # bare_plot removes the axis labels

        """ Run 3D voxel traversal to generate binary mask of L-System network """
        vol = process_network(update, tVol=tissueVolume).astype(np.uint8)
        vessels.append(vol)
        
        x, y, z = np.where(vol > 0)
        
        scio.savemat(
            f"vessels/sim_{timestamp}.mat", 
            {"sim": vol, 
             "x": x.astype(np.uint16), 
             "y": y.astype(np.uint16), 
             "z":z.astype(np.uint16), 
             "properties": properties
            }
        )
        
        pbar.update()

    except SyntaxError as e:
        print(f"Leading 0 error: {e}")
        continue
    except ValueError as e:
        print(f"Division by 0 error: {e}")
        continue

pbar.close()

  0%|          | 0/20 [00:00<?, ?it/s]

Leading 0 error: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<string>, line 1)
Leading 0 error: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<string>, line 1)
Leading 0 error: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<string>, line 1)


In [ ]:
# image = vessels[1]

In [ ]:
# image.shape

In [ ]:
# x,y,z = np.where(image>0)

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(projection="3d")
# ax.scatter(x,y,z,s=0.01)
# plt.show()